In [5]:
import tensorflow as tf
from tensorflow import keras 

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt




In [11]:
# define both features and labels dataframes, save as original dfs

df_ori_features = pd.read_csv('heart-disease-values.csv')

df_ori_labels = pd.read_csv('heart-disease-labels.csv')


# define two dfs which we will manipulate

df_features = df_ori_features

df_labels = df_ori_labels

In [ ]:
# merge both dfs (NB we don't need to do this to feed X and y into our model. 
# But it's good to know how to.)



In [ ]:
# define X and y

